In [1]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash_attn


In [2]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer


In [3]:
model_name = "dunzhang/stella_en_1.5B_v5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Load the tokenizer and the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
embedding_model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device).eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/10.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dunzhang/stella_en_1.5B_v5:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

modeling_qwen.py:   0%|          | 0.00/65.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dunzhang/stella_en_1.5B_v5:
- modeling_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

In [23]:
# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc0 = nn.Linear(input_size, 3072)
        self.dropout0 = nn.Dropout(p=0.6)
        self.relu0 = nn.ReLU()
        self.fc01 = nn.Linear(3072, 3072)
        self.dropout01 = nn.Dropout(p=0.6)
        self.relu01 = nn.ReLU()
        self.fc1 = nn.Linear(3072, 768)
        self.dropout1 = nn.Dropout(p=0.6)
        self.relu1 = nn.ReLU()
        #self.fc11 = nn.Linear(768, 768)
        #self.dropout11 = nn.Dropout(p=0.6)
        #self.relu11 = nn.ReLU()
        #self.fc2 = nn.Linear(768, 384)
        #self.dropout2 = nn.Dropout(p=0.6)
        #self.relu2 = nn.ReLU()
        #self.fc21 = nn.Linear(384, 384)
        #self.dropout21 = nn.Dropout(p=0.6)
        #self.relu21 = nn.ReLU()
        self.fc3 = nn.Linear(768, 2)

    def forward(self, x):
        x = self.fc0(x)
        x = self.dropout0(x)
        x = self.relu0(x)
        x = self.fc01(x)
        x = self.dropout01(x)
        x = self.relu01(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu1(x)
        #x = self.fc11(x)
        #x = self.dropout1(x)
        #x = self.relu1(x)
        #x = self.fc2(x)
        #x = self.dropout2(x)
        #x = self.relu2(x)
        #x = self.fc21(x)
        #x = self.dropout21(x)
        #x = self.relu21(x)
        x = self.fc3(x)
        return x


# Load the model
# Change input_size to 1536 to match the saved model's first layer input
#input_size = 1536
model = SimpleNN(input_size)

In [34]:
# Load the model weights
model_path = 'stella_model.pth'  # Path to your model file
# Loading model


model.load_state_dict(torch.load('stella_model.pth', map_location=device))


<ipython-input-34-88bbaea3eab2>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('stella_model.pth', map_location=device))


<All keys matched successfully>

In [35]:
import torch

def predict_real_or_fake(sentence, embedding_model, classifier_model, tokenizer, device):
    # Tokenize and prepare the input
    tokens = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Move tokens to the same device as the models
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    # Get embeddings from the embedding model (e.g., BERT-like model)
    with torch.no_grad():
        embeddings = embedding_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

    # Use the pooled output, mean, or another aggregation to get [batch_size, 1536]
    # Example: Mean pooling (adjust based on your specific model output requirements)
    pooled_embeddings = embeddings.mean(dim=1)  # [batch_size, embedding_size]

    # Pass the processed embeddings to the classifier model
    with torch.no_grad():
        prediction = classifier_model(pooled_embeddings)

    return prediction.argmax().item()


In [36]:
# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run prediction
user_input = input("Enter a sentence: ")
prediction = predict_real_or_fake(user_input, embedding_model, model, tokenizer, device)
print(f"The prediction is: {prediction}")


Enter a sentence: vaccine contains graphen
The prediction is: 1
